# Setting Up a Basic Hadoop Environment for ETL

In this notebook, we'll guide you through setting up a basic Hadoop environment for ETL operations using HDFS, YARN, MapReduce, and Apache Pig. We'll execute commands directly from this notebook using cell magics.

## Section 1: Prerequisites

### 1.1 Operating System

This setup can be performed on Windows, macOS, or Unix-based operating systems. Since Hadoop is designed to run on Unix-like systems, Windows users will need to set up a Unix-like environment using tools like Windows Subsystem for Linux (WSL) or Cygwin.

**For Windows Users**:

**Option 1: Windows Subsystem for Linux (WSL)**

WSL allows you to run a Linux distribution directly on Windows. You can install Ubuntu from the Microsoft Store.

-   Enable WSL:

Open PowerShell as Administrator and run:

`Enable-WindowsOptionalFeature -Online -FeatureName Microsoft-Windows-Subsystem-Linux`

Restart your computer when prompted.

- Install Ubuntu:

Download and install Ubuntu from the [Microsoft Store](https://apps.microsoft.com/detail/9nblggh4msv6?rtc=1&hl=en-us&gl=US).

- Launch Ubuntu and Update Packages:



In [ ]:
%%bash
sudo apt-get update -V
sudo apt-get upgrade -y

**Option 2: Cygwin**

Cygwin provides a Unix-like environment on Windows.

- Download and install Cygwin from www.cygwin.com.
- During installation, select the packages required (e.g., OpenSSH, OpenJDK).
- Proceed with the instructions in the Cygwin terminal.

**Note**: Using WSL is recommended for simplicity and compatibility.

**For MacOS Users:**

Most Unix commands are compatible with macOS. You may need to install some packages using Homebrew.

- Install Homebrew (if not already installed)

In [ ]:
%%bash
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

- Update Homebrew

In [ ]:
%%bash
brew update

### 1.2 Install Java Development Kit

Hadoop requires Java to run. We'll install JDK 8.

**Check if Java is Already Installed**

Let's verify if Java is already installed on your system.

In [ ]:
%%bash
java -version

**Install JDK 8**
If Java is not installed or you need to install JDK 8, follow the instructions below:

**For Windows Users (Using WSL or Cygwin):**

- Install OpenJDK 8:

In [ ]:
%%bash
sudo apt-get install -y openjdk-8-jdk

- Set Java Environment Variables:

In [ ]:
%%bash
echo "export JAVA_HOME=$(readlink -f /usr/bin/java | sed 's:/bin/java::')" >> ~/.bashrc
source ~/.bashrc

**For macOS Users:**

- Install OpenJDK 8 Using Homebrew:

In [ ]:
%%bash
brew tap homebrew/cask-versions
brew install --cask adoptopenjdk8

- Set Java Environment Variables:

In [ ]:
%%bash
echo "export JAVA_HOME=$(/usr/libexec/java_home -v1.8)" >> ~/.bash_profile
source ~/.bash_profile

**Verify Java Installation**

After installation, confirm that Java is correctly installed:

In [ ]:
%%bash
java -version

You should see output similar to:

`java version "1.8.0_xxx"`

`Java(TM) SE Runtime Environment (build 1.8.0_xxx)`

`Java HotSpot(TM) 64-Bit Server VM (build xx.x-bxx, mixed mode)`


### 1.3 Configure SSH

Hadoop uses SSH for communication between nodes, even in a single-node setup. We'll set up passwordless SSH access to `localhost`.

First, ensure that SSH is installed:

**For Windows Users (Using WSL or Cygwin):**

In [ ]:
%%bash
sudo apt-get install -y openssh-server openssh-client

**For macOS Users:**

SSH is typically pre-installed on macOS. Verify with:

In [ ]:
%%bash
ssh -V

**Generate SSH Keys**

Generate a new SSH key pair without a passphrase:

In [ ]:
%%bash
ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa

**Configure Authorized Keys**

Add your public key to the list of authorized keys:

In [ ]:
%%bash
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod 600 ~/.ssh/authorized_keys

**Start SSH Service (If Necessary)**

For Windows Users (Using WSL or Cygwin):

In [ ]:
%%bash
sudo service ssh start

**Test SSH Connection to localhost**

Verify that you can SSH to localhost without a password:

In [ ]:
%%bash
ssh -o StrictHostKeyChecking=no localhost echo "SSH connection established."

If successful, you should see:

`SSH connection established.`

## Section 2: Install Hadoop

In this section, we'll download and install Hadoop on your system. We'll also set up the necessary environment variables. The instructions are tailored for Windows (using WSL or Cygwin), macOS, and Unix-based systems.

### 2.1 Download Hadoop

We'll download the latest stable release of Hadoop from the Apache website.

**Determine the Latest Stable Version**

Please verify the latest version from the Apache Hadoop Releases page.

In [ ]:
%%bash
# Replace with the latest version number
HADOOP_VERSION=3.3.1

**Download Hadoop**

For All Users:

In [ ]:
%%bash
# Download Hadoop
wget https://downloads.apache.org/hadoop/common/hadoop-$HADOOP_VERSION/hadoop-$HADOOP_VERSION.tar.gz -P ~/

Note: If wget is not installed, you can install it using:

- For Ubuntu/WSL/Cygwin:

In [ ]:
%%bash
sudo apt-get install -y wget

- For macOS:

In [ ]:
%%bash
brew install wget

### 2.2 Extract Hadoop

Unpack the downloaded tarball and move it to a convenient location.

In [ ]:
%%bash
# Extract Hadoop
tar -xzvf ~/hadoop-$HADOOP_VERSION.tar.gz -C ~/

# Move Hadoop to /usr/local/hadoop (may require sudo)
sudo mv ~/hadoop-$HADOOP_VERSION /usr/local/hadoop

**Note for Windows Users**: If you encounter permission issues, you might need to adjust the permissions or run the commands with appropriate privileges.

### 2.3 Set Environment Variables

We'll set up the environment variables required for Hadoop to run properly.

**For All Users:**

Edit your shell profile file to include Hadoop and Java environment variables.

**Determine Your Shell Profile File**

- For Bash shell: ~/.bashrc or ~/.bash_profile
- For Zsh shell (common on macOS): ~/.zshrc

For this guide, we'll assume you're using ~/.bashrc.

**Edit the Shell Profile**

Append the following lines to your shell profile file.

In [ ]:
%%bash
# Define shell profile file
SHELL_PROFILE=~/.bashrc

# Append Hadoop environment variables
echo "
# Hadoop Environment Variables
export HADOOP_HOME=/usr/local/hadoop
export PATH=\$PATH:\$HADOOP_HOME/bin:\$HADOOP_HOME/sbin
" >> \$SHELL_PROFILE

**Set JAVA_HOME Environment Variable**

We need to set JAVA_HOME so that Hadoop knows where Java is installed.

**For Ubuntu/WSL/Cygwin Users:**

In [ ]:
%%bash
# Append JAVA_HOME to shell profile
echo "
export JAVA_HOME=\$(readlink -f /usr/bin/java | sed 's:/bin/java::')
" >> \$SHELL_PROFILE

**For macOS Users:**

In [ ]:
%%bash
# Append JAVA_HOME to shell profile
echo "
export JAVA_HOME=\$(/usr/libexec/java_home -v1.8)
" >> \$SHELL_PROFILE

**Apply the Changes**

Reload your shell profile to apply the environment variable changes.

In [ ]:
%%bash
source ~/.bashrc

**Note for macOS Users**: If you edited ~/.bash_profile or ~/.zshrc, use that file in the source command.

### 2.4 Verify Hadoop Installation

In [ ]:
%%bash
hadoop version

You should see output similar to:

`Hadoop 3.3.1`

`Source code repository https://github.com/apache/hadoop -r ...`

`Compiled by ... on ...`

`Compiled with protoc 3.x.x`

`From source with checksum ...`

`This command was run using /usr/local/hadoop/share/hadoop/common/hadoop-common-3.3.1.jar`


If you see this output, Hadoop is installed successfully.

## Section 3: Configure HDFS

### 3.1 Edit `core-site.xml`

The `core-site.xml` file contains configuration settings for Hadoop core, such as the default filesystem.

**Locate the `core-site.xml` File**

The file is located in the Hadoop configuration directory:

In [ ]:
$HADOOP_HOME/etc/hadoop/core-site.xml

**Create a Backup (Optional)**

It's a good practice to create a backup before modifying configuration files.

In [ ]:
%%bash
cp $HADOOP_HOME/etc/hadoop/core-site.xml $HADOOP_HOME/etc/hadoop/core-site.xml.backup

**Edit `core-site.xml`**

We'll add configuration settings to specify the default filesystem.

In [ ]:
%%bash
cat <<EOL > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
   <property>
       <name>fs.defaultFS</name>
       <value>hdfs://localhost:9000</value>
   </property>
</configuration>
EOL

### 3.2 Edit `hdfs-site.xml`

The hdfs-site.xml file contains settings specific to HDFS, such as replication factor.

**Create a backup (optional)**

In [ ]:
%%bash
cp $HADOOP_HOME/etc/hadoop/hdfs-site.xml $HADOOP_HOME/etc/hadoop/hdfs-site.xml.backup

**Edit `hdfs-site.xml`**

We'll set the replication factor to 1 since we're setting up a single-node cluster.

In [ ]:
%%bash
cat <<EOL > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
   <property>
       <name>dfs.replication</name>
       <value>1</value>
   </property>
</configuration>
EOL

### 3.3 Format the NameNode

Before starting HDFS, we need to format the NameNode.

In [ ]:
%%bash
hdfs namenode -format

You should see output indicating that the NameNode has been formatted successfully.

## Section 4: Configure YARN and MapReduce

In this section, we'll configure YARN (Yet Another Resource Negotiator) and MapReduce. YARN is Hadoop's cluster resource management system, and MapReduce is the programming model for data processing. We'll edit the necessary configuration files to enable these components.

### 4.1 Edit `yarn-site.xml`

The `yarn-site.xml` file contains configuration settings for YARN.

**Locate the `yarn-site.xml` File**

The file is located in the Hadoop configuration directory:

In [ ]:
$HADOOP_HOME/etc/hadoop/yarn-site.xml

**Create a Backup (Optional)**

It's a good practice to create a backup before modifying configuration files.

In [ ]:
%%bash
cp $HADOOP_HOME/etc/hadoop/yarn-site.xml $HADOOP_HOME/etc/hadoop/yarn-site.xml.backup

**Edit `yarn-site.xml`**

We'll add configuration settings to specify the YARN NodeManager auxiliary services and enable the MapReduce shuffle service.

In [ ]:
%%bash
cat <<EOL > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<configuration>
    <!-- Site specific YARN configuration properties -->
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
</configuration>
EOL

### 4.2 Edit `mapred-site.xml`

The `mapred-site.xml` file contains configuration settings for MapReduce.

**Create the `mapred-site.xml` File**

If the file does not exist, create it by copying the template.

In [ ]:
%%bash
cp $HADOOP_HOME/etc/hadoop/mapred-site.xml.template $HADOOP_HOME/etc/hadoop/mapred-site.xml

**Create a Backup (Optional)**

In [ ]:
%%bash
cp $HADOOP_HOME/etc/hadoop/mapred-site.xml $HADOOP_HOME/etc/hadoop/mapred-site.xml.backup

**Edit mapred-site.xml**

We'll configure MapReduce to run on YARN.

In [ ]:
%%bash
cat <<EOL > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<configuration>
    <!-- Site specific MapReduce configuration properties -->
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>
</configuration>
EOL

### 4.3 Verify Configuration Files

You can verify the contents of the configuration files to ensure they have been set correctly.

**View `yarn-site.xml`**

In [ ]:
%%bash
cat $HADOOP_HOME/etc/hadoop/yarn-site.xml

**View `mapred-site.xml`**

In [ ]:
%%bash
cat $HADOOP_HOME/etc/hadoop/mapred-site.xml

**Explanation:**

- `yarn-site.xml`: We specified the yarn.nodemanager.aux-services property and set its value to mapreduce_shuffle. This enables the shuffle service, which is necessary for MapReduce jobs to run properly on YARN.
- `mapred-site.xml`: We set the mapreduce.framework.name property to yarn, indicating that MapReduce should use YARN as its resource manager.

By configuring these files, we've set up Hadoop to use YARN for resource management and prepared the system to run MapReduce jobs.

## Section 5: Start Hadoop Services

In this section, we'll start the Hadoop services, including HDFS (Hadoop Distributed File System) and YARN (Yet Another Resource Negotiator). We'll also verify that the services are running correctly by accessing their web interfaces.

### 5.1 Starrt HDFS (NameNode and DataNode)

We need to start the HDFS daemons: the NameNode and DataNode.

In [ ]:
%%bash
start-dfs.sh

**Note**: If you encounter a "command not found" error, ensure that $HADOOP_HOME/bin and $HADOOP_HOME/sbin are in your PATH. Alternatively, use the full path to the script:

In [ ]:
%%bash
$HADOOP_HOME/sbin/start-dfs.sh

**Expected Output**

You should see output indicating that the NameNode and DataNode are starting:

In [ ]:
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/hadoop/logs/hadoop-yourusername-namenode-yourhostname.out
localhost: starting datanode, logging to /usr/local/hadoop/logs/hadoop-yourusername-datanode-yourhostname.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/hadoop/logs/hadoop-yourusername-secondarynamenode-yourhostname.out

### 5.2 Start YARN (ResourceManager and NodeManager)

Next, we'll start the YARN daemons: the ResourceManager and NodeManager.

In [ ]:
%%bash
start-yarn.sh

Alternatively, use the full path:

In [ ]:
%%bash
$HADOOP_HOME/sbin/start-yarn.sh

**Expected Output**

You should see output indicating that the ResourceManager and NodeManager are starting:

In [ ]:
starting resourcemanager, logging to /usr/local/hadoop/logs/yarn-yourusername-resourcemanager-yourhostname.out
localhost: starting nodemanager, logging to /usr/local/hadoop/logs/yarn-yourusername-nodemanager-yourhostname.out

### 5.3 Verify Running Services

We can verify that the Hadoop services are running by checking the web interfaces provided by HDFS and YARN.

**5.3.1 HDFS NameNode Web Interface**

- URL: http://localhost:9870

Open a web browser and navigate to http://localhost:9870. You should see the HDFS NameNode status page.

**5.3.2 YARN ResourceManager Web Interface**
- URL: http://localhost:8088
Open a web browser and navigate to http://localhost:8088. You should see the YARN ResourceManager status page.

**Note**: If you cannot access these pages, ensure that your firewall settings allow connections to these ports.

### 5.4 Verify Hadoop Processes

You can check if the Hadoop daemons are running by listing Java processes.

In [ ]:
%%bash
jps

**Expected Output**

You should see output similar to:

`ResourceManager`

`NameNode`

`DataNode`

`NodeManager`

`SecondaryNameNode`

`Jps`


### 5.5 Troubleshooting

**Common Issues**

- Environment Variables Not Set: Ensure that `JAVA_HOME` and `HADOOP_HOME` are set correctly and that `$HADOOP_HOME/bin` and `$HADOOP_HOME/sbin` are in your `PATH`.
- SSH Issues: If you encounter SSH connection issues, ensure that passwordless SSH is set up correctly (refer back to Section 1.3).
- Permission Denied Errors: Make sure you have the necessary permissions to start Hadoop services. Avoid running Hadoop as the root user; instead, use a regular user account.

**Log Files**

Check the Hadoop log files for detailed error messages:

- HDFS Logs: Located in `$HADOOP_HOME/logs/`, files like `hadoop-yourusername-namenode-yourhostname.out`.
- YARN Logs: Located in `$HADOOP_HOME/logs/`, files like `yarn-yourusername-resourcemanager-yourhostname.out`.

**Explanation**:

- Starting HDFS Services: The `start-dfs.sh` script starts the HDFS daemons, which include the NameNode, DataNode, and SecondaryNameNode.
- Starting YARN Services: The `start-yarn.sh` script starts the YARN daemons, including the ResourceManager and NodeManager.
- Web Interfaces: Hadoop provides web interfaces to monitor the cluster's status. The NameNode UI shows the status of HDFS, and the ResourceManager UI shows the status of YARN and running applications.
- Process Verification: Using the `jps` command, we can list all Java processes to confirm that the necessary Hadoop daemons are running.

## Section 6: Run a Sample MapReduce Job

In this section, we'll run a sample MapReduce job to ensure that our Hadoop setup is functioning correctly. We'll use the built-in WordCount example that comes with Hadoop. This example will count the frequency of words in a set of input files.

### 6.1 Prepare the HDFS Directory Structure

We need to create directories in HDFS to store our input files and output results.

#### 6.1.1 Create a User Directory in HDFS

Replace yourusername with your actual username if necessary. Alternatively, you can use the whoami command to dynamically get your username.

In [ ]:
%%bash
hdfs dfs -mkdir -p /user/$(whoami)

Verify the Directory Creation

In [ ]:
%%bash
hdfs dfs -ls /user

You should see your username listed in the output.

### 6.2 Copy Sample Files to HDFS

We'll use the Hadoop configuration files as sample input data for the WordCount example.

In [ ]:
%%bash
hdfs dfs -put $HADOOP_HOME/etc/hadoop/*.xml /user/$(whoami)/

Verify Files in HDFS

In [ ]:
%%bash
hdfs dfs -ls /user/$(whoami)/

You should see a list of .xml files that were copied to your HDFS directory.

### 6.3 Run the WordCount MapReduce Job

We'll execute the WordCount example using the sample files we just uploaded.

#### 6.3.1 Identify the Hadoop MapReduce Examples JAR

First, find the exact version of the Hadoop MapReduce examples JAR file.

In [ ]:
%%bash
ls $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-*.jar

This command will output the full path to the examples JAR, such as:

In [ ]:
/usr/local/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.1.jar

#### 6.3.2 Execute the WordCount Job

Run the WordCount job using the identified JAR file.

In [ ]:
%%bash
hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-*.jar wordcount /user/$(whoami) /user/$(whoami)/output

**Explanation**:

- Input Path: `/user/$(whoami)` (the directory containing your input files)
- Output Path: `/user/$(whoami)/output` (the directory where the results will be stored)

**Expected Output**

The terminal will display logs showing the progress of the MapReduce job. Look for lines indicating that the job has completed successfully.

### 6.4 View the Output Results

After the job completes, we can check the output stored in HDFS.

#### 6.4.1 List the Output Directory

In [ ]:
%%bash
hdfs dfs -ls /user/$(whoami)/output

You should see files like part-r-00000, which contain the word count results.

#### 6.4.2 Display the Results

In [ ]:
%%bash
hdfs dfs -cat /user/$(whoami)/output/part-r-00000

This command will output the word counts. You should see something like:

In [ ]:
Configuration    2
Filesystem       3
Hadoop           5
MapReduce        1
...

### 6.5 Verify Job Status on YARN ResourceManager UI

You can also verify that the job ran successfully by checking the YARN web interface.

- URL: http://localhost:8088

Navigate to this URL in your web browser. Under the "**Finished Applications**" section, you should see your WordCount job listed.

### 6.6 Clean Up the Output Directory (Optional)

If you want to run the job again, you need to remove the existing output directory; otherwise, Hadoop will throw an error because the output directory already exists.

In [ ]:
%%bash
hdfs dfs -rm -r /user/$(whoami)/output

**Explanation**:

- **Creating Directories**: We created necessary directories in HDFS to organize our data.

- **Uploading Data**: We uploaded sample files to HDFS to serve as input for our MapReduce job.

- **Running the Job**: We executed the WordCount example, which processed the input files and produced word counts.

- **Viewing Results**: We retrieved the output from HDFS and displayed it.

- **Web Interface**: The YARN ResourceManager UI provides a graphical interface to monitor and manage jobs.

## Section 7: Install Apache Pig

In this section, we'll install Apache Pig, a high-level platform for creating programs that run on Hadoop. Pig is designed to simplify the processing and analysis of large datasets. We'll download and install Pig, set the necessary environment variables, and ensure it's correctly configured to work with our Hadoop setup.

### 7.1 Download Apache Pig

We'll download the latest stable version of Apache Pig from the Apache website.

**Determine the Latest Stable Version**

Please check the Apache Pig Releases page to confirm the latest version.

In [ ]:
%%bash
# Replace with the latest Pig version number if necessary
PIG_VERSION=0.17.0

**Download Apache Pig**

In [ ]:
%%bash
# Download Apache Pig
wget https://downloads.apache.org/pig/pig-$PIG_VERSION/pig-$PIG_VERSION.tar.gz -P ~/

**Note**: If wget is not installed, you can install it using:

- For Ubuntu/WSL/Cygwin:

In [ ]:
%%bash
sudo apt-get install -y wget

- For macOS:

In [ ]:
%%bash
brew install wget

### 7.2 Extract Apache Pig

Unpack the downloaded tarball and move it to a suitable location.

In [ ]:
%%bash
# Extract Pig
tar -xzvf ~/pig-$PIG_VERSION.tar.gz -C ~/

# Move Pig to /usr/local/pig (may require sudo)
sudo mv ~/pig-$PIG_VERSION /usr/local/pig

**Note for Windows Users**: If you encounter permission issues, you might need to adjust permissions or run the commands with administrative privileges.

### 7.3 Set Up Environment Variables

We need to set up environment variables so that the system recognizes Pig commands.

**Edit Your Shell Profile**

Append the following lines to your shell profile file.

In [ ]:
%%bash
# Define shell profile file
SHELL_PROFILE=~/.bashrc  # Use ~/.bash_profile or ~/.zshrc if appropriate

# Append Pig environment variables
echo "
# Pig Environment Variables
export PIG_HOME=/usr/local/pig
export PATH=\$PATH:\$PIG_HOME/bin
" >> \$SHELL_PROFILE

**Apply the Changes**

Reload your shell profile to apply the environment variable changes.

In [ ]:
%%bash
source ~/.bashrc

**Note for macOS Users**: If you edited ~/.bash_profile or ~/.zshrc, replace ~/.bashrc with the appropriate file in the source command.

### 7.4 Verify Apache Pig Installation

Let's confirm that Pig is installed and configured correctly.

In [ ]:
%%bash
pig -version

You should see output similar to:

In [ ]:
Apache Pig version 0.17.0 (r1744791) compiled Mar 3 2016, 14:42:23

### 7.5 Configure Pig (Optional)

By default, Pig runs in **MapReduce mode**, which requires a running Hadoop cluster. Since we have Hadoop configured, no additional setup is needed for Pig to run in this mode.

If you wish to run Pig in **local mode** (without Hadoop), you can specify the -x local option when running Pig commands.

**Explanation**:

- **Downloading Pig**: We downloaded the specified version of Apache Pig to ensure compatibility with our Hadoop installation.
- **Setting Environment Variables**: By updating PIG_HOME and PATH, we make Pig commands accessible from any directory.
- **Verification**: Running pig -version checks that Pig is installed and can interact with Hadoop.
- **Configuration**: Pig is set to run in MapReduce mode by default, which is suitable for our setup.

## Section 8: Run an ETL Process Using Apache Pig

In this section, we'll perform an ETL (Extract, Transform, Load) operation using Apache Pig. We'll use Pig Latin scripts to process data stored in HDFS. This exercise will demonstrate how Pig simplifies complex data transformations and makes it easier to work with large datasets on Hadoop.

### 8.1 Prepare Sample Data

We'll create a sample data file and upload it to HDFS to serve as the input for our Pig script.

#### 8.1.1 Create a Sample Data File

Let's create a file named student_data.txt with the following content:

In [ ]:
%%bash
cat <<EOL > ~/student_data.txt
1,John,Doe,85
2,Jane,Smith,92
3,Bob,Johnson,76
4,Alice,Williams,89
5,Tom,Brown,95
EOL

**Explanation**:

Each line represents a student's record with the following fields:

- `id`: Student ID (integer)
- `first_name`: First name (string)
- `last_name`: Last name (string)
- `score`: Exam score (integer)

#### 8.1.2 Upload the Data File to HDFS

We'll copy the student_data.txt file from the local filesystem to HDFS.

In [ ]:
%%bash
hdfs dfs -mkdir -p /user/$(whoami)/pig_data
hdfs dfs -put ~/student_data.txt /user/$(whoami)/pig_data/

Verify the file in HDFS

In [ ]:
%%bash
hdfs dfs -ls /user/$(whoami)/pig_data/

### 8.2 Write a Pig Script

We'll write a Pig script named etl.pig to perform the ETL operation.

#### 8.2.1 Create the Pig Script

In [ ]:
%%bash
cat <<EOL > ~/etl.pig
-- Load data from HDFS
student_data = LOAD '/user/$(whoami)/pig_data/student_data.txt' USING PigStorage(',')
                 AS (id:int, first_name:chararray, last_name:chararray, score:int);

-- Filter records with score greater than 80
high_scores = FILTER student_data BY score > 80;

-- Group data by score
grouped_data = GROUP high_scores BY score;

-- Calculate average score (optional step)
avg_score = FOREACH grouped_data GENERATE group AS score, COUNT(high_scores) AS count;

-- Store the result back to HDFS
STORE high_scores INTO '/user/$(whoami)/pig_output/high_scores' USING PigStorage(',');

-- Store the average score (optional)
STORE avg_score INTO '/user/$(whoami)/pig_output/avg_score' USING PigStorage(',');
EOL

**Explanation**:

- **Load** Statement: Reads data from HDFS using PigStorage with a comma as the delimiter.
- **Filter** Statement: Filters records where score > 80.
- **Group** Statement: Groups the filtered data by score.
- **Foreach** Statement: Calculates the count of students for each score (optional step).
- **Store** Statements: Saves the filtered data and average scores back to HDFS.

#### 8.2.2 Review the Pig Script (Optional)

In [ ]:
%%bash
cat ~/etl.pig

### 8.3 Execute the Pig Script

We'll run the Pig script using the pig command.

In [ ]:
%%bash
pig ~/etl.pig

**Note**: Ensure that Hadoop services are running before executing the Pig script. If not, refer back to Section 5 to start Hadoop services.

**Expected Output**

The terminal will display logs showing the progress of the Pig script execution. Look for lines indicating successful completion:

In [ ]:
Success!
Job Stats (time in seconds):
JobId   Maps   Reduces  MaxMapTime  MinMapTime  AvgMapTime  MaxReduceTime  MinReduceTime  AvgReduceTime  Alias        Feature  Outputs
job_...     1        1         ...         ...         ...            ...            ...            ...  student_data  MAP_ONLY  ...
job_...     1        1         ...         ...         ...            ...            ...            ...  high_scores   HADOOP   ...

Input(s):
Successfully read ... records

Output(s):
Successfully stored ... records

Counters:
Total records written : ...
Total bytes written : ...


### 8.4 Verify the Results

We'll check the output stored in HDFS to ensure our ETL process worked as expected.

#### 8.4.1 List the Output Directories

In [ ]:
%%bash
hdfs dfs -ls /user/$(whoami)/pig_output/high_scores/
hdfs dfs -ls /user/$(whoami)/pig_output/avg_score/


You should see part-m-00000 files in each directory.

#### 8.4.2 Display the High Scores

In [ ]:
%%bash
hdfs dfs -cat /user/$(whoami)/pig_output/high_scores/part-m-00000

Expected Output:

In [ ]:
2,Jane,Smith,92
4,Alice,Williams,89
5,Tom,Brown,95

#### 8.4.3 Display the Average Scores (Optional)

In [ ]:
%%bash
hdfs dfs -cat /user/$(whoami)/pig_output/avg_score/part-r-00000


Expected Output:

In [ ]:
89,1
92,1
95,1

**Explanation**:

Each line represents a score and the count of students who achieved it.

### 8.5 Clean Up (Optional)

If you wish to rerun the Pig script, you need to remove the existing output directories to avoid errors.

In [ ]:
%%bash
hdfs dfs -rm -r /user/$(whoami)/pig_output/

**Explanation**:

- **Data Preparation**: We created a sample dataset representing student records and uploaded it to HDFS.
- **Pig Script**: We wrote a Pig script to load the data, filter students with scores greater than 80, group the data, and calculate counts.
- **Execution**: We ran the Pig script, which executed MapReduce jobs under the hood.
- **Verification**: We retrieved and displayed the output from HDFS to confirm that the ETL process was successful.

**Key Concepts**:

- **Pig Latin**: The scripting language used by Apache Pig, which simplifies writing data transformation operations.
- **ETL Process**: Extracting data from a source, transforming it according to business logic, and loading it into a destination.
- **MapReduce Jobs**: Pig scripts are internally converted into MapReduce jobs for execution on Hadoop.

## Section 9: Cleanup and Shutdown

In this final section, we'll cover how to properly shut down the Hadoop services and clean up any temporary files or directories created during this exercise. Regular cleanup helps maintain system performance and frees up resources.

### 9.1 Stop Hadoop Services

To gracefully shut down the Hadoop services, we'll stop both HDFS and YARN daemons.

#### 9.1.1 Stop YARN Services

In [ ]:
%%bash
stop-yarn.sh

Alternatively, if the command is not found:

In [ ]:
%%bash
$HADOOP_HOME/sbin/stop-yarn.sh

Expected Output

You should see messages indicating that the ResourceManager and NodeManager are stopping:

In [ ]:
stopping resourcemanager
localhost: stopping nodemanager

#### 9.1.2 Stop HDFS Services

In [ ]:
%%bash
stop-dfs.sh

Alternatively:

In [ ]:
%%bash
$HADOOP_HOME/sbin/stop-dfs.sh

**Expected Output**

You should see messages indicating that the NameNode and DataNode are stopping:

In [ ]:
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode

### 9.2 Verify that Services Have Stopped

You can check that the Hadoop daemons are no longer running by listing Java processes.

In [ ]:
%%bash
jps

**Expected Output**

You should only see the Jps process listed:

In [ ]:
Jps

### 9.3 Clean Up Temp Files (Optional)

During Hadoop's operation, temporary files and directories are created. You may choose to clean these up to free disk space.

#### 9.3.1 Remove HDFS Data Directories

If you want to completely remove all data stored in HDFS (including the NameNode and DataNode data), you can delete the Hadoop data directories. By default, these are located in /tmp/hadoop-$(whoami).

**Warning**: This will delete all data in HDFS.

In [ ]:
%%bash
rm -rf /tmp/hadoop-$(whoami)*

#### 9.3.2 Remove HDFS Files and Directories

If you only want to remove the files and directories created in HDFS during this exercise:

In [ ]:
%%bash
hdfs dfs -rm -r /user/$(whoami)/*

### 9.4 Remove Sample Data Files (Optional)

If you wish to remove the sample data files from your local filesystem:

In [ ]:
%%bash
rm ~/student_data.txt
rm ~/etl.pig

### 9.5 Reset Environment Variables (Optional)

If you want to remove the environment variables set during this setup:

#### 9.5.1 Edit Your Shell Profile

Open your shell profile file (e.g., ~/.bashrc, ~/.bash_profile, or ~/.zshrc) in a text editor and remove the lines related to HADOOP_HOME, PIG_HOME, and their additions to the PATH.

Alternatively, you can use the following commands to remove them automatically:

In [ ]:
%%bash
sed -i '/# Hadoop Environment Variables/,+2d' ~/.bashrc
sed -i '/# Pig Environment Variables/,+2d' ~/.bashrc

**Note**: The sed command may differ between systems, and the above command works for GNU sed. On macOS, you might need to use sed -i '' instead.

### 9.6 Restart Your Shell

After making changes to your shell profile, restart your terminal or reload the shell configuration:

In [ ]:
%%bash
source ~/.bashrc

### 9.7 Additional Cleanup (Optional)

If you installed any software specifically for this exercise and wish to remove it:

#### 9.7.1 Remove Hadoop and Pig Directories

In [ ]:
%%bash
sudo rm -rf /usr/local/hadoop
sudo rm -rf /usr/local/pig

#### 9.7.2 Remove Downloaded Tarballs

In [ ]:
%%bash
rm ~/hadoop-*.tar.gz
rm ~/pig-*.tar.gz


**Explanation**:

- **Stopping Services**: It's important to properly stop Hadoop services to ensure all processes terminate gracefully.
- **Cleaning Up**: Removing temporary files and directories helps maintain system health and frees up disk space.
- **Environment Variables**: Resetting environment variables cleans up your shell environment if you no longer need Hadoop and Pig commands accessible globally.
- **Data Preservation**: Be cautious when deleting data directories to avoid accidental loss of important data.

## Conclusion

Congratulations! You've completed setting up a basic Hadoop environment and performed ETL operations using Apache Pig. You've learned how to:

- Install and configure Hadoop components: HDFS, YARN, and MapReduce.
- Start and stop Hadoop services.
- Run a sample MapReduce job.
- Install and use Apache Pig for data processing.
- Clean up your environment after use.

This exercise has provided hands-on experience with Hadoop's ecosystem, and you're now better equipped to explore more advanced features and large-scale data processing tasks.

Next Steps:

- Explore More Pig Scripts: Try modifying the Pig script to perform different transformations or analyses on your data.
- Learn Hive: Consider setting up Apache Hive for SQL-like querying of large datasets on Hadoop.
- Set Up a Multi-Node Cluster: If you're interested in scaling up, explore setting up a multi-node Hadoop cluster to handle larger workloads.